In [2]:
import torch
model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)
model.eval()

Using cache found in C:\Users\User/.cache\torch\hub\pytorch_vision_v0.5.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

def load_train_data(train_path, train_batch_size, size, shuffle = True):
    transformers = transforms.Compose([
                    transforms.Resize(size),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    set_data = datasets.ImageFolder(root = train_path, transform = transformers)
    return DataLoader(set_data, batch_size = train_batch_size, shuffle = shuffle, num_workers = 1)

def load_test_data(test_path, test_batch_size, size, shuffle = True):
    transformers = transforms.Compose([transforms.Resize((size, size)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    set_data = datasets.ImageFolder(root = test_path, transform = transformers)
    return DataLoader(set_data, batch_size = test_batch_size, shuffle = shuffle, num_workers = 1)

In [4]:
import torch
# from Models import ConvNet
# from Load import load_data

from tqdm import tqdm


EPOCHS = 10
Learning_rate = 0.001
Batch_size = 64
L2_rate = 0
image_size = 256

data_size = 9017
num_batches = data_size//Batch_size
num_classes = 7

Train_path = "C:/Users/User/Desktop/Astghik/Skin_cancer/data/train/"


data_loader = load_train_data(Train_path, Batch_size, image_size)


def train():
    model.train()

    crossentropy = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=Learning_rate)

    for epoch in range(EPOCHS):
        epoch_loss = 0
        epoch_acc = 0
        for X, y in tqdm(data_loader):

            optimizer.zero_grad()

            out = model(X) 

            
            loss = crossentropy(out, y)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item() 

            predictions = torch.argmax(out, 1)
            epoch_acc += torch.sum(predictions==y).item()

        epoch_loss = epoch_loss/num_batches
        epoch_acc = epoch_acc/data_size
        print(f"Epoch {epoch}:")
        print("ACC:", epoch_acc, "LOSS:", epoch_loss)

        torch.save(model.state_dict(),
                   f"C:/Users/User/Desktop/Astghik/Skin_cancer/saved_models/resnet18/ResNet_{epoch}.model")


In [4]:
train()

  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

Epoch 0:
ACC: 0.7109903515581679 LOSS: 1.0173933938145638


  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

Epoch 1:
ACC: 0.7685482976599756 LOSS: 0.631894037766116


  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

...

Epoch 7:
ACC: 0.8717977154264168 LOSS: 0.34673515994633947


  0%|                                                                                          | 0/141 [00:00<?, ?it/s]

Epoch 8:
ACC: 0.8846623045358767 LOSS: 0.3135347042764936


100%|████████████████████████████████████████████████████████████████████████████████| 141/141 [28:14<00:00, 12.02s/it]

Epoch 9:
ACC: 0.899412221359654 LOSS: 0.2820188319044454


In [11]:
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm

#model = ConvNet(7)

state_dict = torch.load("C:/Users/User/Desktop/Astghik/Skin_cancer/saved_models/resnet18/ResNet_6.model")
model.load_state_dict(state_dict)

data_size = 998
test_batch_size = 64
size = 256

test_path = "C:/Users/User/Desktop/Astghik/Skin_cancer/data/test/"

data_loader = load_test_data(test_path, test_batch_size, size, shuffle=False)

def test():
    model.eval()
    acc = 0
    y_hat = []
    y_true = []
    for X, y in tqdm(data_loader):
        
        out = model(X)
        
        predictions = torch.argmax(out, 1)
        acc += torch.sum(predictions == y).item()
        y_hat.append(predictions)
        y_true.append(y)
        
    y_hat = torch.cat(y_hat)
    y_true = torch.cat(y_true)
    acc = acc/data_size
    print(acc)
    print(classification_report(y_hat, y_true))

In [10]:
test() #6

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [01:10<00:00,  4.38s/it]


0.8106212424849699
              precision    recall  f1-score   support

           0       0.31      0.67      0.43        15
           1       0.80      0.62      0.70        66
           2       0.72      0.54      0.62       144
           3       0.36      0.50      0.42         8
           4       0.43      0.69      0.53        70
           5       0.92      0.90      0.91       684
           6       0.71      0.91      0.80        11

    accuracy                           0.81       998
   macro avg       0.61      0.69      0.63       998
weighted avg       0.83      0.81      0.82       998

